In [ ]:
import nltk
import numpy as np
import pandas as pd
import networkx as nx
tweets = pd.read_csv("/content/drive/MyDrive/deptweet_GNN/corpus.csv")

tweets.head()

,Unnamed: 0,id,tweet,label,parsed
0,0,1.622571e+10,Whenever it gets close to my bday I always go ...,non-depressed,Whenever it gets close to my bday I always go ...
1,1,2.951188e+10,". Yeah, I can see that. My stuff is not depres...",non-depressed,". Yeah, I can see that. My stuff is not depres..."
2,2,5.290000e+17,My focus has been scattered lately between dif...,mild,My focus has been scattered lately between dif...
3,3,6.540000e+17,"I don't always think it equals depression, tho...",non-depressed,"I don't always think it equals depression, tho..."
4,4,8.600000e+17,That show was ghastly! It had such an air of w...,non-depressed,That show was ghastly! It had such an air of w...


In [ ]:
tweets.shape

(40191, 5)

In [ ]:
tweets = tweets.iloc[:20000,:]

In [ ]:
tweets.shape

(20000, 5)

In [ ]:
tweets_list = tweets["tweet"].values.tolist()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# import the necessary libraries
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# define the preprocessing function
def preprocess_tweet(tweet):
    # remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    # remove user mentions
    tweet = re.sub(r'@\w+', '', tweet)
    # tokenize the tweet
    words = word_tokenize(tweet)
    # remove stop words
    words = [word for word in words if word.lower() not in stopwords.words('english')]
    # lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # join the words back into a string
    tweet = ' '.join(words)
    return tweet

# preprocess the tweets
preprocessed_tweets = [preprocess_tweet(tweet) for tweet in tweets_list]


In [ ]:
# import the necessary libraries
import spacy

# load the spacy model
nlp = spacy.load('en_core_web_sm')

# define the entity extraction function
def extract_entities(tweet):
    doc = nlp(tweet)
    entities = []
    for ent in doc.ents:
        if ent.label_ in ['PERSON', 'ORG', 'GPE']:
            entities.append(ent.text)
    return entities

# extract entities from the tweets
entities_list = [extract_entities(tweet) for tweet in preprocessed_tweets]


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
import nltk
import networkx as nx

# Function to extract named entities and relationships from a text
def extract_entities_and_relationships(text):
    # Tokenize the text and extract the part-of-speech tags
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)

    # Extract the named entities
    entities = nltk.ne_chunk(pos_tags, binary=True)

    # Extract the relationships between entities
    relationships = []
    for i in range(len(tokens)):
        if pos_tags[i][1] == 'NN':
            subject = tokens[i]
        if pos_tags[i][1] == 'IN':
            object = tokens[i+1]
            relationships.append((subject, object))

    return entities, relationships

# Function to construct a knowledge graph from the extracted entities and relationships
def construct_knowledge_graph(entities, relationships):
    # Create a directed graph
    G = nx.DiGraph()

    # Add the entities as nodes to the graph
    for entity in entities:
        if hasattr(entity, 'label') and entity.label:
            G.add_node(entity[0][0], label=entity.label())
        else:
            G.add_node(entity[0], label='NAMED_ENTITY')

    # Add the relationships as edges to the graph
    for relationship in relationships:
        G.add_edge(relationship[0], relationship[1])

    return G

with open('/content/drive/MyDrive/deptweet_GNN/tweets_data.csv','r')as f:
  text = f.read()
# Extract the entities and relationships from the text
entities, relationships = extract_entities_and_relationships(text)

# Construct the knowledge graph from the extracted entities and relationships
G = construct_knowledge_graph(entities, relationships)


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
!pip install 'scipy>=1.8'
!pip install 'networkx<2.7'

In [ ]:
import os

plt.figure(figsize=(12, 12))

pos =  nx.spring_layout(G, k=0.5)

nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos=pos , font_size=12)
plt.show()